# 🚀 Auto Manual Review
---
### The code is in the proper order.
### 👉 **Run each cell according to the instructions above each code cell.**

## 💻 Environment Setup

To run this notebook smoothly, we recommend the following environment:

### 🧠 IDE
- **[Visual Studio Code](https://code.visualstudio.com/)** (VS Code)
  A lightweight, powerful editor that supports Jupyter notebooks out of the box.

### 🧩 Required Extensions
- **Jupyter** extension (published by Microsoft)
  - Go to Extensions `(Ctrl+Shift+X)` → Search for `Jupyter` → Install.
- (Optional) **Python** extension (also by Microsoft) for syntax highlighting and Python support.

### 🧪 Python Environment
- Python version **3.9+** recommended.
- Use `venv`, `conda`, or your preferred environment manager to isolate dependencies.

### 🔁 Kernel Instructions
Once you open the notebook:
1. Click the top-right **kernel selector** (it may say “Python 3” or “Select Kernel”).
2. Choose the environment where you've installed your requirements.
3. If no environment appears, make sure it’s activated and Python is installed.

## ▶️ How to Use

This project is designed as a Jupyter Notebook, which runs Python code in cells. Here's how to interact with it:

### 🧾 Opening the Notebook
1. Launch **VS Code**.
2. Open the folder containing this project.
3. Open the `.ipynb` file (`auto_manual_review.ipynb` or similar).

### 🚀 Running Cells
- **Click** on a cell to select it.
- **Run a cell** by pressing:
  - `Shift + Enter` — runs the cell and moves to the next.
  - `Ctrl + Enter` — runs the cell but keeps the focus on it.
  - You can also use the **▶️ Run** button in the top bar.

### 📌 Important Notes
- **Run cells in order.** The notebook is designed to be executed from top to bottom.
- **Don't skip setup cells**, especially those that handle imports, functions, and cookie authentication.
- Output will appear directly below each cell when run.

# 📦 Cell 0: First-Time Setup 🚀
---
This cell ensures that all the required libraries are installed and ready to go for running the rest of the notebook.

✅ **What It Does**:
- Automatically checks for missing libraries:
  - `selenium`
  - `beautifulsoup4`
  - `pandas`
  - `requests`
  - `pyclip`
- Installs any missing ones using `pip`.

📌 **When to Use**:
- The very first time you run this notebook.
- Or if you’ve reinstalled Python or created a fresh environment.

🧰 **Outcome**:
All dependencies are ready — you're all set to move forward!

In [1]:
# 🚀 First-Time Setup: Check and Install Required Packages

import subprocess
import sys

# List of required packages
required_packages = ["selenium", "beautifulsoup4", "pandas", "requests", "pyclip"]

def install_package(package):
    """Install package using pip."""
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Try importing each package, install if not found
for package in required_packages:
    try:
        __import__(package.split('==')[0])
        print(f"✅ {package} is already installed.")
    except ImportError:
        print(f"📦 {package} not found. Installing...")
        install_package(package)
        print(f"✅ {package} installed successfully!")

print("\n🎉 Environment is ready!")

✅ selenium is already installed.
📦 beautifulsoup4 not found. Installing...
✅ beautifulsoup4 installed successfully!
✅ pandas is already installed.
✅ requests is already installed.
✅ pyclip is already installed.

🎉 Environment is ready!


# 🛠️ Cell 1: Imports
---
Imports all necessary Python libraries for the script to function.

✅ **What It Includes**:
- Automation (`selenium`)
- Web scraping (`BeautifulSoup`)
- Data handling (`pandas`)
- API requests (`requests`)
- Clipboard access (`pyclip`)
- File and JSON operations

📌 **When to Use**:
Run this **once** every time you open the notebook.

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from IPython.display import Markdown, display
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pyclip
import os
import json

# 🧠 Cell 2: Function Definitions
---
This cell defines all the core functions used in the notebook.

✅ **What It Does**:
- Sets up reusable code blocks to:
  - Load and save cookies
  - Open Chrome and log in
  - Fetch reference data
  - Check registration and match percentages
  - Display results cleanly

📌 **When to Use**:
Run once at the beginning of each session.
Must be run **before any other code cells** that depend on these functions.

In [3]:
def save_cookies_to_file(cookies, filename="cookies.json"):
    with open(filename, "w") as f:
        json.dump(cookies, f)

def load_cookies_from_file(filename="cookies.json"):
    if os.path.exists(filename):
        with open(filename, "r") as f:
            return json.load(f)
    return None

def is_session_valid(session):
    """Check if kog.tw session is still valid"""
    try:
        resp = session.get("https://kog.tw/player_edit.php?player=")
        # Player edit page without player param usually redirects if not logged in
        return resp.status_code == 200 and "inputEmail" in resp.text
    except:
        return False

def acquire_cookies():
    """Main cookie acquisition logic"""
    # Try loading cookies from file first
    cookies = load_cookies_from_file()
    if cookies:
        print("⌛ Loaded cookies from file. Verifying...")
        session = requests.Session()
        session.cookies.update(cookies)
        if is_session_valid(session):
            print("🍪 Cookies are still valid!")
            return cookies
        else:
            print("❌ Cookies expired or invalid. Need to log in again.")

    # Otherwise, manual login
    chrome_options = Options()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome(options=chrome_options)
    driver.get("https://kog.tw")
    print("Browser opened. Please log in manually...")

    while True:
        user_input = input("Type 'done' after logging in or 'exit' to cancel: ")
        if user_input.lower() == 'done':
            break
        if user_input.lower() == 'exit':
            driver.quit()
            return None
        print("⌛ Waiting for login...")

    cookies = {
        c['name']: c['value']
        for c in driver.get_cookies()
        if c['name'] in ('PHPSESSID', 'cf_clearance')
    }
    driver.quit()

    save_cookies_to_file(cookies)
    print("💾 Saved new cookies to file.")
    return cookies

def manual_cookie_fallback():
    print("Alternative method:")
    print("1. Visit https://kog.tw in Chrome")
    print("2. Open DevTools (F12 → Network tab)")
    print("3. Refresh and copy a request's 'Cookie' header")
    cookie_header = input("Paste cookie header here: ")
    return dict(pair.split("=", 1) for pair in cookie_header.split("; "))

def scrape_player_data(session, ref_number):
    url = f"https://kog.tw/player_migration.php?ref={ref_number}"
    response = session.get(url)

    if response.status_code != 200:
        print(f"❗ [{response.status_code}] ERROR: Unable to fetch the page.")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    headers = soup.find_all("h1")

    if len(headers) <= 1:
        print("❗ ERROR: Not enough h1 elements on the page.")
        return []

    table = headers[1].find_next("table")
    return [
        {
            "name": row.find_all("td")[0].text.strip(),
            "finishes": row.find_all("td")[1].text.strip()
        }
        for row in table.find_all("tr")[1:]
        if len(row.find_all("td")) == 2
    ]

def check_player_info(session, player_name):
    url = f"https://kog.tw/player_edit.php?player={player_name}"
    try:
        response = session.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")
        email_input = soup.find("input", {"name": "inputEmail"})

        if not email_input:
            return "❌ UNREGISTERED (NO EMAIL FIELD)"

        email = email_input.get("value", "")
        return "✅ REGISTERED" if email.strip() else "❌ UNREGISTERED"

    except Exception as e:
        print(f"Error checking {player_name}: {str(e)}")
        return "ERROR"

def check_player_ip(session, player_name, ip_address):
    """Check a player using IP address via kog.tw API."""
    try:
        url = "https://kog.tw/api.php?automated=1"

        payload = {
            "type": "user/admin/check_player",
            "data": {
                "playername": player_name,
                "playerip": ip_address
            }
        }

        headers = {
            "Content-Type": "application/json"
        }

        response = session.post(url, json=payload, headers=headers)
        response.raise_for_status()

        data = response.json()

    except Exception as e:
        print(f"❗ Error checking IP: {str(e)}")
        return None

def check_all_players(session, player_data, ip_address):
    """Check all players for registration status and IP match percentage."""
    results = []

    for player in player_data:
        name = player['name']
        finishes = player['finishes']

        # Check if player is registered
        status = check_player_info(session, name)

        # Default percentage = None
        percentage = None

        # If registered, check IP percentage
        if status == "REGISTERED":
            percentage = check_player_ip(session, name, ip_address)

        results.append({
            'name': name,
            'status': status,
            'finishes': finishes,
            'match_percentage': percentage
        })

    return results

def generate_output(review_name, player_data):
    output = (
        f"## Manual review is for: `{review_name}`\n"
        "Have you registered or completed a map with one or more of the following names?\n\n"
    )
    output += "\n".join(f"- `{p['name']}`" for p in player_data)
    output += (
        "\n\n### Please elaborate your case:\n"
        "- If you already registered one of these names, why are you trying to register a new name?\n"
        "- If you just finished with one of these names, please do not finish maps for other names besides the one associated with your account.\n"
        "- If you did not register or finish maps for any of these names, please confirm that you did not register or finish any maps for these names.\n\n"
        "*While you are waiting for us, make sure to familiarize yourself with our [#kog-rulebook](https://discord.com/channels/342003344476471296/978628693389885490)*"
    )
    return output

def display_results_table(results):
    """Display table nicely in Jupyter"""
    if not results:
        print("❌ No results to display")
        return

    pd.set_option('display.max_rows', None)

    df = pd.DataFrame(results)

    display(df)

def display_full_results(results):
    """Display the full results as a DataFrame."""
    df = pd.DataFrame(results)
    display(df)  # In Jupyter this will show a nice clean table

# 🍪 Cell 3: Grab Cookies
---
Handles user login and manages session cookies.

🔐 **How It Works**:
1. Checks if `cookies.json` already exists and is still valid.
2. If not, opens a Chrome window and prompts you to login at [kog.tw](https://kog.tw).
3. After logging in, you’ll type `done` to save cookies.

💾 **Outcome**:
Creates or updates `cookies.json` for future automated requests.

📌 **Important**:
This is only needed **if no cookies exist** or **your session expires**.

In [4]:
cookies = acquire_cookies()
if not cookies:
    cookies = manual_cookie_fallback()

⌛ Loaded cookies from file. Verifying...
❌ Cookies expired or invalid. Need to log in again.
Browser opened. Please log in manually...
💾 Saved new cookies to file.


# 🔎 Cell 4: Locate Review with Reference Number and Check Usernames
---
Looks up review data using a **reference number** and checks linked usernames.

🧠 **What It Does**:
- Finds the account linked to the reference number.
- Extracts all alternate usernames.
- Automatically skips the first name (it’s the same as the original account).
- Checks each one for **registration status**.

📌 **When to Use**:
After your cookies are working and you're ready to start a lookup.

In [28]:
session = requests.Session()
session.cookies.update(cookies)

ref_number = input("Enter the reference number: ")
player_data = scrape_player_data(session, ref_number)

review_name = player_data[0]['name']
player_data = player_data[1:]

if not player_data:
    print("❌ No player data found.")
else:
    print("✅ Player data found!")


print(f"⌛ [{ref_number}] Checking {len(player_data)} players...")
results = []

for player in player_data:
    status = check_player_info(session, player['name'])
    results.append({
        'name': player['name'],
        'status': status,
        'finishes': player['finishes']
    })

registered_name = [
    r for r in results if r['status'] == "✅ REGISTERED"
]

print("✅ Check complete!")

✅ Player data found!
⌛ [ref1714057] Checking 5 players...
✅ Check complete!


# 📝 Cell 5: Generate Copy/Paste Text
---
Generates a formatted message with the results from **Cell 4**, ready for pasting into chat.

🧾 **What It Includes**:
- Each checked username
- Their registration status
- Formatted neatly for readability

📋 **Bonus**:
Automatically copies the message to your clipboard for fast sharing.

📌 **When to Use**:
Right after checking usernames. Makes reporting results super quick!

In [31]:
full_output = generate_output(review_name, player_data)
display(Markdown(full_output))

try:
    pyclip.copy(full_output)
    print("📋 Copied to clipboard.")
except:
    print("❌ Could not copy to clipboard (pyclip error).")


## Manual review is for: `Motto`
Have you registered or completed a map with one or more of the following names?

- `KedyDeluxe`
- `KlipLipp`
- `Honda City`
- `Kurumi`
- `Tahacimke`

### Please elaborate your case:
- If you already registered one of these names, why are you trying to register a new name?
- If you just finished with one of these names, please do not finish maps for other names besides the one associated with your account.
- If you did not register or finish maps for any of these names, please confirm that you did not register or finish any maps for these names.

*While you are waiting for us, make sure to familiarize yourself with our [#kog-rulebook](https://discord.com/channels/342003344476471296/978628693389885490)*

📋 Copied to clipboard.


# 🗂️ Cell 6: Collect Registered Usernames Only
---
Pulls out just the usernames that came back as ✅ REGISTERED in Cell Y and stores them in a tidy list for anything you need next.

**🧾 What It Does**:
- Scans the full results list
- Filters for entries whose status is ✅ REGISTERED
- Builds a clean, duplicate-free list called registered_names

**⚙️ Under the Hood**: Uses a one-liner list-comprehension (or a loop, if you prefer) so you always get the freshest set of confirmed names.

**📋 Bonus**: Returns the list in the same order the users were checked—handy if you need to keep things sequential.

**📌 When to Run It**: Immediately after your status-checking cell finishes. Run this, then hand the list off to downstream tasks like generating the manual-review template or syncing to your database.

In [30]:
registered_output = generate_output(review_name, registered_name)
display(Markdown(registered_output))

try:
    pyclip.copy(registered_output)
    print("📋 Copied to clipboard.")
except:
    print("❌ Could not copy to clipboard (pyclip error).")

## Manual review is for: `Motto`
Have you registered or completed a map with one or more of the following names?

- `KedyDeluxe`
- `KlipLipp`
- `Kurumi`
- `Tahacimke`

### Please elaborate your case:
- If you already registered one of these names, why are you trying to register a new name?
- If you just finished with one of these names, please do not finish maps for other names besides the one associated with your account.
- If you did not register or finish maps for any of these names, please confirm that you did not register or finish any maps for these names.

*While you are waiting for us, make sure to familiarize yourself with our [#kog-rulebook](https://discord.com/channels/342003344476471296/978628693389885490)*

📋 Copied to clipboard.


# 📊 Cell 7: Display Status Report *(Optional)*
---

This cell takes the results from **Cell 5** — which contains the registration status of each username — and displays them in a neatly formatted table using **Pandas**.

✅ **Purpose**:
To quickly scan and verify which usernames are registered or unregistered, along with their associated map finishes.

🧠 **What It Does**:
- Converts the list of results (name, status, finishes) into a `DataFrame`.
- Outputs the table with improved formatting in Jupyter Notebook (not just plain text).

📌 **When to Use**:
- After you’ve run **Cell 5** and want a clearer visual overview.
- Optional step for better readability — does not affect any later cells.

🖥️ **Output Example**:
| name       | status             | finishes |
|------------|--------------------|----------|
| `Player1`  | ✅ REGISTERED       | 12       |
| `Player2`  | ❌ UNREGISTERED     | 0        |

In [8]:
display_results_table(results)

,name,status,finishes
0,KedyDeluxe,✅ REGISTERED,1
1,KlipLipp,✅ REGISTERED,11
2,Honda City,❌ UNREGISTERED,1
3,Kurumi,✅ REGISTERED,22
4,Tahacimke,✅ REGISTERED,7


# 🗂️ Cell 8: Generate Verification Prompt *(Optional)*
---
Creates a detailed **follow-up message** to send to a user who has map finishes on multiple registered usernames.

🧠 **What It Does**:
- Lists all **registered usernames** from the current result.
- Adds a polite prompt asking the user for clarification.
- Formats the message in Markdown with backticks and emphasis.
- Attempts to **copy** the entire message to your clipboard.

📋 **Message Example**:
```markdown
### You have map finishes on the following registered usernames: `User1`, `User2`, `User3`

*Are you aware that only one account is allowed per player?*

What can you tell me about these accounts?
```

💡 **Clipboard Support**:
If successful, it prints: `📋 Copied to clipboard.`
If there's an issue with `pyclip`, it prints an error instead.

📌 **When to Use**:
After verifying registrations — this cell is useful for **manual outreach** or moderation follow-ups.

In [21]:
verification = f"### You have map finishes on the following registered usernames: "
verification += ", ".join(f"`{p['name']}`" for p in results if p['status'] == "✅ REGISTERED")

verification += f"\n*Are you aware that only one account is allowed per player?*"

verification += (f"\n\nWhat can you tell me about these accounts?")

print(verification)

try:
    pyclip.copy(verification)
    print("\n📋 Copied to clipboard.")
except:
    print("\n❌ Could not copy to clipboard (pyclip error).")

### You have map finishes on the following registered usernames: `paulwalker`
*Are you aware that only one account is allowed per player?*

What can you tell me about these accounts?

📋 Copied to clipboard.


# 🧮 Cell 9: Display Full Report *(Optional)*
---
Expands the previous report by checking **match percentages** for all registered usernames.

📊 **What It Adds**:
- Percentage match for each name (via the API).
- Appends this data to the same table from **Cell 6**.

🎯 **Useful For**:
- Deeper analysis
- Flagging suspicious overlaps

📌 **When to Use**:
After you’ve checked usernames and want full details in one place.

In [10]:
ip_address = input("Enter IP address to check: ")

if ip_address:
    full_results = check_all_players(session, player_data, ip_address)
    display_full_results(full_results)

else:
    print("❌ No IP address found.")

No IP address found.
